# E2-5 Tic Tac Toe Wiser - Invincible

This is a game, in which AI starts playing from fixed winning position.
The second move is based on a rule: 
- if possible, occupy the corners
- otherwise, the middle

## Initialization

In [1]:
# Initialize the players and signs
EMPTY = '.'
AI = 'X'
HUMAN = 'O'

In [2]:
# print the board, leave an empty lines and spaces for visibility
def print_board(board):
    print(" ")
    print(' '.join(board[:3]))
    print(' '.join(board[3:6]))
    print(' '.join(board[6:]))
    print(" ")

In [3]:
# Initialize the board
board = EMPTY * 9
print_board(board)

 
. . .
. . .
. . .
 


In [4]:
# Define all possible winning combinations
win_cases = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [0, 3, 6],
    [1, 4, 7],
    [2, 5, 8],
    [0, 4, 8],
    [2, 4, 6]
]

##  Supporting Functions

#### Function improved
AI is interested in winning and save options, with fixed start at 9 and 7.
First move is always in a corner; second move is in the opposite corner if possible, otherwise in the middle.

In [5]:
# Get all available save moves on the board in a list, fix the first two
def all_possible_moves(board, sign):
    if sign == AI:
        empty = board.count(EMPTY)
        if empty == 9:
            return [sign + EMPTY * 8]
        elif empty == 7:
            return [
                board[:8] + sign if board[8] == EMPTY else
                board[:4] + sign + board[5:]
            ]
    move_list = []
    for i, v in enumerate(board):
        if v == EMPTY:
            new_board = board[:i] + sign + board[i+1:]
            move_list.append(new_board)
            if game_won_by(new_board) == AI:
                return [new_board]
    if sign == AI:
        safe_moves = []
        for move in move_list:
            if not can_win(move, HUMAN):
                safe_moves.append(move)
        return safe_moves if len(safe_moves) > 0 else move_list[0:1]
    else:
        return move_list

In [6]:
# A winning game is if any of win-cases occurs
def game_won_by(board):
    for i in win_cases:
        if board[i[0]] == board[i[1]] == board[i[2]] != EMPTY:
            # win-case
            return board[i[0]]
    return EMPTY

In [7]:
# Before make a move, iterate through the options and check if some wins
def can_win(board, sign):
    next_moves = all_possible_moves(board, sign)
    for next_move in next_moves:
        if game_won_by(next_move) == sign:
            return True
    return False

## Play The Game

### Game Moves

#### Human move

In [8]:
# Human move approach still the same
def human_move(board, row, column):
    # get the index of the cell the user selected: 2D -> 1D 
    index = 3 * (row - 1) + (column - 1)
    #  if this cell is empty, make the user move, otherwise do nothing
    if board[index] == EMPTY:
        # place HUMAN sign on board[index]
        return board[:index] + HUMAN + board[index+1:]
    return board

#### AI move: ai_move() is improved
Save preffered moves

In [9]:
# AI makes a move from all available SAFE moves
from random import choice
def ai_move(board):
    new_boards = all_possible_moves(board, AI)
    for new_board in new_boards:
        if game_won_by(new_board) == AI:
            return new_board
    safe_moves = []
    for new_board in new_boards:
        if not can_win(new_board, HUMAN):
            safe_moves.append(new_board)
    if len(safe_moves) > 0: 
        return choice(safe_moves) 
    return new_boards[0]           

In [10]:
# Play the game
def game():
    # start from empty board
    board = EMPTY * 9
    empty_cell_count = 9
    end_flag = False
    
    while empty_cell_count > 0 and not end_flag:       
        # Player AI (always odd number of options)
        if empty_cell_count % 2 == 1:
            board = ai_move(board)
        else:
            # Human player
            row = int(input('Enter row: '))
            col = int(input('Enter column: '))
            board = human_move(board, row, col)
            
        # Print current board status    
        print_board(board)
        
        # Check if someone wins already, update the flag
        end_flag = game_won_by(board) != EMPTY
        
        # Count how many empty cells left
        empty_cell_count = board.count(EMPTY)      
        # empty_cell_count = sum(1 for cell in board if cell == EMPTY_SIGN)
     
    print('Game ended. Winner: ', game_won_by(board))

In [23]:
# Run the game
game()

 
X . .
. . .
. . .
 
Enter row: 1
Enter column: 2
 
X O .
. . .
. . .
 
 
X O .
. . .
. . X
 
Enter row: 2
Enter column: 2
 
X O .
. O .
. . X
 
 
X O .
. O .
. X X
 
Enter row: 3
Enter column: 1
 
X O .
. O .
O X X
 
 
X O X
. O .
O X X
 
Enter row: 2
Enter column: 3
 
X O X
. O O
O X X
 
 
X O X
X O O
O X X
 
Game ended. Winner:  .


## Game Analysis

We want to know how many are the possible combinations of moves and how many times each player can win the game.

In [24]:
# all moves for all possible states for this player
def all_moves(board_list, sign):
    move_list = []
    for board in board_list:
        move_list.extend(all_possible_moves(board, sign))
    return move_list

In [13]:
# All wins of each player separated in two new lists, draws remains in the old list
def player_wins(move_list, ai_wins, human_wins):
    for board in move_list:
        won_by = game_won_by(board)
        if won_by == AI:
            ai_wins.append(board)
            move_list.remove(board)
        elif won_by == HUMAN:
            human_wins.append(board)
            move_list.remove(board)

In [14]:
# At each step count the available moves
def count_possibilities():
    board = EMPTY * 9
    move_list = [board]
    ai_wins = []
    human_wins = []
    for i in range(9):
        print('Step ' + str(i) + ': possible moves = ' + str(len(move_list)))
        sign = AI if i % 2 == 0 else HUMAN
        move_list = all_moves(move_list, sign)
        player_wins(move_list, ai_wins, human_wins)
    print('First player wins: ' + str(len(ai_wins)))
    print('Second player wins: ' + str(len(human_wins)))
    print('Draw', str(len(move_list)))
    print('Total', str(len(ai_wins) + len(human_wins) + len(move_list)))

In [15]:
count_possibilities()

Step 0: possible moves = 1
Step 1: possible moves = 1
Step 2: possible moves = 8
Step 3: possible moves = 8
Step 4: possible moves = 48
Step 5: possible moves = 38
Step 6: possible moves = 108
Step 7: possible moves = 76
Step 8: possible moves = 90
First player wins: 128
Second player wins: 0
Draw 60
Total 188
